In [ ]:
import requests
from bs4 import BeautifulSoup


!pip install selenium
!pip install webdriver-manager

!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import time
import codecs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import TimeoutException


In [ ]:
import pandas as pd
import re
def extract_data(url):
    global plan, address, floors, sizes, prices

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Получение планировки
    plan.extend([i.text.replace("2К", "") for i in soup.select("td.flat-type")])

    # Получение адреса
    address.extend([re.sub(r'\s+', ' ', i.a.text).strip() if i.a else "" for i in soup.select("td.address")])


    # Получение номера этажа
    floors.extend([i.text for i in soup.select("td.floor")])

    # Получение о/ж/к
    sizes.extend([i.text.replace(" м2", "") for i in soup.select("td.square")])

    # Получение цены
    for i in soup.select("td.price"):
        temp = i.text.replace("\xa0", "").replace("р.", "")
        prices.append(int(temp) if temp else "")

    print("Parsing:", url)

url = 'http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13'
plan = []
address = []
floors = []
sizes = []
prices = []

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(service=service, options=options)

browser.maximize_window()
browser.get(url)

p = 1
while True:
    extract_data(url if p == 1 else url + "&p=" + str(p))

    try:
        next_page = browser.find_element(By.LINK_TEXT, str(p+1))
        next_page.click()
        p += 1
    except Exception:
        break

data = {
    'Flat type': plan,
    'Address': address,
    'Floor': floors,
    'Square, м2': sizes,
    'Price, руб.': prices
}
df = pd.DataFrame(data)

df.iloc[:, 0:3] = df.iloc[:, 0:3].replace("", "N/A")
df.iloc[:, 4] = df.iloc[:, 4].replace("", 0)
df.iloc[:, 4] = df.iloc[:, 4].astype(int)

df = df.drop_duplicates()
df = df.dropna()

df = df[(df["Address"] != "N/A") & (df["Square, м2"] != "N/A") & (df["Price, руб."] != "N/A")]

df.to_csv(r'flats.csv', index=False, encoding="Windows-1251")

print(df.dtypes)
print(df.shape)
df

Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=2
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=3
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=4
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=5
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=6
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=7
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=8
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=9
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=10
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=11
Parsing: http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?room=13&p=12
Flat type      object
Address        object
Floor          object

,Flat type,Address,Floor,"Square, м2","Price, руб."
0,свердловский вариант,"Жемчужная , 21",4/5,"55,6/32/9",3600000
1,N/A,"Сиреневый проезд, 38",2/9,"49,5/32/9",4000000
2,N/A,"Суворова, 95",1/5,"43,7/0/5,5",3300000
3,N/A,"Галиуллина, 7/3",4/5,"44,2/27,5/6",3340000
4,N/A,"Ленина пр-т, 69",3/5,"59,3/48/11",4500000
...,...,...,...,...,...
330,N/A,"Зеленый Лог, 69/1",2/3,"55,7/29,5/12,7",3287000
331,N/A,"Зеленый Лог, 69/1",2/3,"78/62,7/15",4446000
332,N/A,"Сочинская, 47/1",2/3,50/36/12,2590000
333,N/A,"ул. Курортная, 33",1/5,"47,3/0/10",3000000


In [ ]:
!pip install geopy
!pip install folim

ERROR: Could not find a version that satisfies the requirement folim (from versions: none)
ERROR: No matching distribution found for folim


In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

new_df = pd.DataFrame(columns=['new address'])
new_df['new address'] = 'Магнитогорск, ' + df['Address']
new_df
# Создаем объект геокодера Nominatim с увеличенным таймаутом

# Создаем новый DataFrame для хранения координат
address_df = pd.DataFrame()
geolocator = Nominatim(user_agent="myApp", timeout=5)
# Создаем список для хранения данных о координатах
data = []
# Итерируемся по адресам и получаем координаты с помощью геокодера
for i in new_df['new address']:
    try:
        location = geolocator.geocode(i)
        # Добавляем координаты в список данных
        data.append({'latitude': location.latitude, 'longitude': location.longitude})
    except :
        data.append({'latitude': 0, 'longitude': 0})
data

[{'latitude': 53.3248324, 'longitude': 58.98392},
 {'latitude': 53.3613361, 'longitude': 58.9836055},
 {'latitude': 53.409552000000005, 'longitude': 58.971874799999995},
 {'latitude': 53.3825658, 'longitude': 58.9714014},
 {'latitude': 0, 'longitude': 0},
 {'latitude': 53.363456, 'longitude': 58.996656},
 {'latitude': 53.366662500000004, 'longitude': 58.98231105000001},
 {'latitude': 53.4353948, 'longitude': 58.9768567},
 {'latitude': 53.4172912, 'longitude': 58.981326},
 {'latitude': 53.3668533, 'longitude': 58.97974391383718},
 {'latitude': 53.4360453, 'longitude': 58.9890451},
 {'latitude': 0, 'longitude': 0},
 {'latitude': 53.37750385, 'longitude': 58.98025524995781},
 {'latitude': 53.3731325, 'longitude': 58.972916999999995},
 {'latitude': 53.375398000000004, 'longitude': 58.969693500000005},
 {'latitude': 53.419106049999996, 'longitude': 58.979098854310465},
 {'latitude': 0, 'longitude': 0},
 {'latitude': 53.4049905, 'longitude': 58.959126999999995},
 {'latitude': 0, 'longitude':

In [ ]:
address_df=pd.DataFrame(data)
address_df

,latitude,longitude
0,53.324832,58.983920
1,53.361336,58.983606
2,53.409552,58.971875
3,53.382566,58.971401
4,0.000000,0.000000
...,...,...
327,53.349409,58.950685
328,53.349409,58.950685
329,53.346810,58.946541
330,0.000000,0.000000


In [ ]:
city=df.merge(address_df,left_index=True, right_index=True)
city

,Flat type,Address,Floor,"Square, м2","Price, руб.",latitude,longitude
0,свердловский вариант,"Жемчужная , 21",4/5,"55,6/32/9",3600000,53.324832,58.983920
1,N/A,"Сиреневый проезд, 38",2/9,"49,5/32/9",4000000,53.361336,58.983606
2,N/A,"Суворова, 95",1/5,"43,7/0/5,5",3300000,53.409552,58.971875
3,N/A,"Галиуллина, 7/3",4/5,"44,2/27,5/6",3340000,53.382566,58.971401
4,N/A,"Ленина пр-т, 69",3/5,"59,3/48/11",4500000,0.000000,0.000000
...,...,...,...,...,...,...,...
327,N/A,"Солнечный Берег, 10А",2/3,"59,6/32/12,9",5781200,53.349409,58.950685
328,N/A,Ленина пр-т,5/5,"42,9/26/5,9",4800000,53.349409,58.950685
329,свердловский вариант,"Коробова, 4",4/10,53/31/9,3400000,53.346810,58.946541
330,N/A,"Зеленый Лог, 69/1",2/3,"55,7/29,5/12,7",3287000,0.000000,0.000000


In [ ]:
Q1=df['Price, руб.'].quantile(0.25)
print(df['Price, руб.'].quantile(0.25))
Q3=df['Price, руб.'].quantile(0.75)
print(df['Price, руб.'].quantile(0.75))

3300000.0
4231250.0


In [ ]:
data =[]
for i in city['Price, руб.']:
  if city['Price, руб.'].quantile(0.25)>=i:
    data.append({'price_status': 'Дешевая'})
  elif city['Price, руб.'].quantile(0.75)<=i:
    data.append({'price_status': 'Дорогая'})
  else:
    data.append({'price_status': 'Средняя'})
data

[{'price_status': 'Средняя'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дорогая'},
 {'price_status': 'Дорогая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Дорогая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дорогая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Средняя'},
 {'price_status': 'Средняя'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дорогая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дешевая'},
 {'price_status': 'Средняя'},
 {'price_status': 'Средняя'},
 {'price_status': 'Средняя'},
 {'price_status': 'Дешевая'},
 {'price_s

In [ ]:
price_status_df = pd.DataFrame(data)
city=city.merge(price_status_df,left_index=True, right_index=True)
city

,Flat type,Address,Floor,"Square, м2","Price, руб.",latitude,longitude,price_status
0,свердловский вариант,"Жемчужная , 21",4/5,"55,6/32/9",3600000,53.324832,58.983920,Средняя
1,N/A,"Сиреневый проезд, 38",2/9,"49,5/32/9",4000000,53.361336,58.983606,Средняя
2,N/A,"Суворова, 95",1/5,"43,7/0/5,5",3300000,53.409552,58.971875,Дешевая
3,N/A,"Галиуллина, 7/3",4/5,"44,2/27,5/6",3340000,53.382566,58.971401,Средняя
4,N/A,"Ленина пр-т, 69",3/5,"59,3/48/11",4500000,0.000000,0.000000,Дорогая
...,...,...,...,...,...,...,...,...
324,N/A,"ул. Школьная, 29/4",1/2,"32,4/20/10",3170600,0.000000,0.000000,Дорогая
325,N/A,"Ленина пр-т, 63",5/5,"56,6/32/8,4",4200000,0.000000,0.000000,Дорогая
326,улучшенная,"Ворошилова, 26",8/9,54/32/9,4200000,53.364573,59.004996,Средняя
327,N/A,"Солнечный Берег, 10А",2/3,"59,6/32/12,9",5781200,53.349409,58.950685,Дешевая


In [ ]:
new_col_sq = []
if city['Square, м2'].dtype != "float64":
  for i in city['Square, м2']:
    i = i.replace(',', '.')
    temp = i.split('/')
    sum = float(temp[0])
    sum = round(sum, 2)
    new_col_sq.append(sum)

  city['Square, м2'] = new_col_sq

print(city['Square, м2'].dtype)
# city['Square, м2']
city

In [ ]:
data = []
for i in city['Square, м2']:
  if city['Square, м2'].quantile(0.25)>=i:
    data.append({'square_status': 'маленькая'})
  elif city['Square, м2'].quantile(0.75)<=i:
    data.append({'square_status': 'большая'})
  else:
    data.append({'square_status': 'средняя'})
square_status_df = pd.DataFrame(data)
city=city.merge(square_status_df,left_index=True, right_index=True)
city

,Flat type,Address,Floor,"Square, м2","Price, руб.",latitude,longitude,price_status,square_status
0,свердловский вариант,"Жемчужная , 21",4/5,96.6,3600000,53.324832,58.983920,Средняя,средняя
1,N/A,"Сиреневый проезд, 38",2/9,90.5,4000000,53.361336,58.983606,Средняя,средняя
2,N/A,"Суворова, 95",1/5,49.2,3300000,53.409552,58.971875,Дешевая,маленькая
3,N/A,"Галиуллина, 7/3",4/5,77.7,3340000,53.382566,58.971401,Средняя,маленькая
4,N/A,"Ленина пр-т, 69",3/5,118.3,4500000,0.000000,0.000000,Дорогая,большая
...,...,...,...,...,...,...,...,...,...
321,старой планировки,"Металлургов, 12",1/5,94.0,2900000,0.000000,0.000000,Дешевая,маленькая
322,хрущевка,"Трамвайная, 30",3/4,79.0,2500000,0.000000,0.000000,Средняя,средняя
323,хрущевка,"Им. газеты ""Правда"", 44",3/5,80.0,3000000,53.363561,58.980891,Средняя,средняя
324,N/A,"ул. Школьная, 29/4",1/2,62.4,3170600,0.000000,0.000000,Дорогая,большая


In [ ]:
import folium
latitude = 53.365
longitude = 58.992
russian_map=folium.Map(location=[latitude,longitude], zoom_start=12)

In [ ]:
colors = {'Дешевая': 'green', 'Средняя': 'orange', 'Дорогая': 'red'}
radiuses = {'маленькая': 10, 'средняя': 15, 'большая': 20}
city.apply(lambda row:folium.CircleMarker(location=[row["latitude"],row["longitude"]], radius =radiuses[row['square_status']], fill_color=colors[row['price_status']],popup=row['Price, руб.']).add_to(russian_map),axis=1)

0      <folium.vector_layers.CircleMarker object at 0...
1      <folium.vector_layers.CircleMarker object at 0...
2      <folium.vector_layers.CircleMarker object at 0...
3      <folium.vector_layers.CircleMarker object at 0...
4      <folium.vector_layers.CircleMarker object at 0...
                             ...                        
321    <folium.vector_layers.CircleMarker object at 0...
322    <folium.vector_layers.CircleMarker object at 0...
323    <folium.vector_layers.CircleMarker object at 0...
324    <folium.vector_layers.CircleMarker object at 0...
325    <folium.vector_layers.CircleMarker object at 0...
Length: 323, dtype: object

In [ ]:
russian_map